In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\tonju\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [4]:
# Mars news URL of page to be scraped
news_url = 'https://redplanetscience.com/'

browser.visit(news_url)

html = browser.html

news_soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve the latest news title and paragraph
# Apply inspect to identify class for news and title and save them to a variable
title = news_soup.find_all('div', class_='content_title')[0].text
paragraph = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(title)
print("--------------------------------------------------------------------")
print(paragraph)

Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch
--------------------------------------------------------------------
A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff.


In [ ]:
browser.quit()